<a href="https://colab.research.google.com/github/kaykaushal/spatial_ds_airpoluttion/blob/main/non_spatial_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NASA Air Quality Challenge: NO2 Prediction

## Data Collection

### Binary Files Load

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path
import random
from typing import Dict, List, Union

from cloudpathlib import S3Path, S3Client
import geopandas as gpd
import pandas as pd

In [ ]:
!pip install geopandas #cloudpathlib #,

     |████████████████████████████████| 1.0 MB 5.6 MB/s 
     |████████████████████████████████| 6.3 MB 33.7 MB/s 
     |████████████████████████████████| 16.7 MB 251 kB/s 


In [ ]:
# Training data with geo location 
train_df = pd.read_csv('/content/drive/MyDrive/Driven_Data/NASA-NO2/Data/location_train_data.csv', index_col=0)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36131 entries, 0 to 36130
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   datetime  36131 non-null  object 
 1   grid_id   36131 non-null  object 
 2   value     36131 non-null  float64
 3   location  36131 non-null  object 
 4   tz        36131 non-null  object 
 5   wkt       36131 non-null  object 
dtypes: float64(1), object(5)
memory usage: 1.9+ MB


In [ ]:
train_df.head()

,datetime,grid_id,value,location,tz,wkt
0,2019-01-01T08:00:00Z,3A3IE,8.695000,Los Angeles (SoCAB),Etc/GMT+8,POLYGON ((-117.8889090614936 34.13085940091023...
1,2019-01-01T08:00:00Z,3S31A,10.496667,Los Angeles (SoCAB),Etc/GMT+8,POLYGON ((-117.9338248256995 33.79558357488509...
2,2019-01-01T08:00:00Z,7II4T,37.208333,Los Angeles (SoCAB),Etc/GMT+8,POLYGON ((-118.0236563541115 33.98201108613195...
3,2019-01-01T08:00:00Z,8BOQH,9.791667,Los Angeles (SoCAB),Etc/GMT+8,POLYGON ((-118.4278982319653 34.01924766510738...
4,2019-01-01T08:00:00Z,A2FBI,4.308333,Los Angeles (SoCAB),Etc/GMT+8,POLYGON ((-117.3948356552278 33.98201108613195...


In [ ]:
#metdata for satellite file
no2_satellite_md = pd.read_csv('/content/drive/MyDrive/Driven_Data/NASA-NO2/Data/no2_satellite_metadata_0AF3h09.csv')
no2_satellite_md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5013 entries, 0 to 5012
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   granule_id    5013 non-null   object
 1   time_start    5013 non-null   object
 2   time_end      5013 non-null   object
 3   product       5013 non-null   object
 4   location      5013 non-null   object
 5   split         5013 non-null   object
 6   us_url        5013 non-null   object
 7   eu_url        5013 non-null   object
 8   as_url        5013 non-null   object
 9   cksum         5013 non-null   int64 
 10  granule_size  5013 non-null   int64 
dtypes: int64(2), object(9)
memory usage: 430.9+ KB


In [ ]:
no2_satellite_md.head()

,granule_id,time_start,time_end,product,location,split,us_url,eu_url,as_url,cksum,granule_size
0,20190101T213357_tropomi_la_0.nc,2019-01-01T20:35:33.000Z,2019-01-01 21:33:57+00:00,tropomi,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,1611277770,332149977
1,20190102T211456_tropomi_la_0.nc,2019-01-02T20:16:33.000Z,2019-01-02 21:14:56+00:00,tropomi,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,1146925070,334150299
2,20190103T205556_tropomi_la_0.nc,2019-01-03T19:57:33.000Z,2019-01-03 20:55:56+00:00,tropomi,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,165673661,334661071
3,20190104T203656_tropomi_la_0.nc,2019-01-04T19:38:33.000Z,2019-01-04 20:36:56+00:00,tropomi,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,2963951339,337713636
4,20190105T201756_tropomi_la_0.nc,2019-01-05T19:19:33.000Z,2019-01-05 20:17:56+00:00,tropomi,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,4029547159,333323724


In [ ]:
no2_satellite_md['product'].value_counts()

tropomi_hires    2750
tropomi          1193
omi              1070
Name: product, dtype: int64

In [ ]:
no2_satellite_md['location'].value_counts()

la           1461
dl           1262
tpe          1220
dl-la-tpe    1070
Name: location, dtype: int64

In [ ]:
no2_satellite_md['split'].value_counts()

train    3156
test     1857
Name: split, dtype: int64

In [ ]:
omi_mdf = no2_satellite_md[no2_satellite_md['product'] == 'omi']
omi_mdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1070 entries, 3943 to 5012
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   granule_id    1070 non-null   object
 1   time_start    1070 non-null   object
 2   time_end      1070 non-null   object
 3   product       1070 non-null   object
 4   location      1070 non-null   object
 5   split         1070 non-null   object
 6   us_url        1070 non-null   object
 7   eu_url        1070 non-null   object
 8   as_url        1070 non-null   object
 9   cksum         1070 non-null   int64 
 10  granule_size  1070 non-null   int64 
dtypes: int64(2), object(9)
memory usage: 100.3+ KB


In [ ]:
omi_mdf.head()

,granule_id,time_start,time_end,product,location,split,us_url,eu_url,as_url,cksum,granule_size
3943,20190101T235959_omi_dl-la-tpe_0.he5,2019-01-01T00:00:00.000Z,2019-01-01 23:59:59+00:00,omi,dl-la-tpe,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,2423440162,11127998
3944,20190102T235959_omi_dl-la-tpe_0.he5,2019-01-02T00:00:00.000Z,2019-01-02 23:59:59+00:00,omi,dl-la-tpe,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,3526775882,11169726
3945,20190103T235959_omi_dl-la-tpe_0.he5,2019-01-03T00:00:00.000Z,2019-01-03 23:59:59+00:00,omi,dl-la-tpe,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,388814420,10937679
3946,20190104T235959_omi_dl-la-tpe_0.he5,2019-01-04T00:00:00.000Z,2019-01-04 23:59:59+00:00,omi,dl-la-tpe,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,1410261098,11141005
3947,20190105T235959_omi_dl-la-tpe_0.he5,2019-01-05T00:00:00.000Z,2019-01-05 23:59:59+00:00,omi,dl-la-tpe,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,1765932191,10819793


In [ ]:
omi_mdf.split.value_counts()

train    671
test     399
Name: split, dtype: int64

In [ ]:
tropomi_mdf = no2_satellite_md[no2_satellite_md['product'] == 'tropomi']
tropomi_mdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1193 entries, 0 to 1192
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   granule_id    1193 non-null   object
 1   time_start    1193 non-null   object
 2   time_end      1193 non-null   object
 3   product       1193 non-null   object
 4   location      1193 non-null   object
 5   split         1193 non-null   object
 6   us_url        1193 non-null   object
 7   eu_url        1193 non-null   object
 8   as_url        1193 non-null   object
 9   cksum         1193 non-null   int64 
 10  granule_size  1193 non-null   int64 
dtypes: int64(2), object(9)
memory usage: 111.8+ KB


In [ ]:
tropomi_mdf.head()

,granule_id,time_start,time_end,product,location,split,us_url,eu_url,as_url,cksum,granule_size
0,20190101T213357_tropomi_la_0.nc,2019-01-01T20:35:33.000Z,2019-01-01 21:33:57+00:00,tropomi,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,1611277770,332149977
1,20190102T211456_tropomi_la_0.nc,2019-01-02T20:16:33.000Z,2019-01-02 21:14:56+00:00,tropomi,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,1146925070,334150299
2,20190103T205556_tropomi_la_0.nc,2019-01-03T19:57:33.000Z,2019-01-03 20:55:56+00:00,tropomi,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,165673661,334661071
3,20190104T203656_tropomi_la_0.nc,2019-01-04T19:38:33.000Z,2019-01-04 20:36:56+00:00,tropomi,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,2963951339,337713636
4,20190105T201756_tropomi_la_0.nc,2019-01-05T19:19:33.000Z,2019-01-05 20:17:56+00:00,tropomi,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,4029547159,333323724


In [ ]:
hires_mdf = no2_satellite_md[no2_satellite_md['product'] == 'tropomi_hires']
hires_mdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2750 entries, 1193 to 3942
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   granule_id    2750 non-null   object
 1   time_start    2750 non-null   object
 2   time_end      2750 non-null   object
 3   product       2750 non-null   object
 4   location      2750 non-null   object
 5   split         2750 non-null   object
 6   us_url        2750 non-null   object
 7   eu_url        2750 non-null   object
 8   as_url        2750 non-null   object
 9   cksum         2750 non-null   int64 
 10  granule_size  2750 non-null   int64 
dtypes: int64(2), object(9)
memory usage: 257.8+ KB


In [ ]:
hires_mdf.head()

,granule_id,time_start,time_end,product,location,split,us_url,eu_url,as_url,cksum,granule_size
1193,20190806T203500_tropomi_hires_la_0.nc,2019-08-06T19:36:35.000Z,2019-08-06 20:35:00+00:00,tropomi_hires,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,98226049,458238869
1194,20190806T221629_tropomi_hires_la_0.nc,2019-08-06T21:18:06.000Z,2019-08-06 22:16:29+00:00,tropomi_hires,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,3731707405,451625521
1195,20190807T201554_tropomi_hires_la_0.nc,2019-08-07T19:17:29.000Z,2019-08-07 20:15:54+00:00,tropomi_hires,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,90599763,456171857
1196,20190807T215723_tropomi_hires_la_0.nc,2019-08-07T20:59:00.000Z,2019-08-07 21:57:23+00:00,tropomi_hires,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,1856867893,452744421
1197,20190808T213817_tropomi_hires_la_0.nc,2019-08-08T20:39:54.000Z,2019-08-08 21:38:17+00:00,tropomi_hires,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,1357264836,453115772


### Satellite data download 

In [ ]:
!pip install cloudpathlib[s3]


     |████████████████████████████████| 132 kB 9.0 MB/s 
     |████████████████████████████████| 8.6 MB 62.5 MB/s 
     |████████████████████████████████| 79 kB 6.9 MB/s 
     |████████████████████████████████| 138 kB 69.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from cloudpathlib import S3Path, S3Client
s3_cli = S3Client(no_sign_request=True)

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
# Download satellite data for tropomi
tropomi_path = S3Path("s3://drivendata-competition-airathon-public-as/no2/train/tropomi/2019/", client=s3_cli)
tropomi_files = list(tropomi_path.rglob("*.nc"))

In [ ]:
fn = tropomi_files[0].fspath

In [ ]:
import netCDF4 as nc

In [ ]:
ds = nc.Dataset(fn)

In [ ]:
ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.7
    institution: KNMI
    source: Sentinel 5 precursor, TROPOMI, space-borne remote sensing, L2
    summary: TROPOMI/S5P NO2 1-Orbit L2 Swath 7x3.5km
    tracking_id: 26a7dcde-bd7b-4f7d-b469-01cef40b796a
    id: S5P_OFFL_L2__NO2____20190101T031859_20190101T050029_06310_01_010202_20190107T050224
    time_reference: 2019-01-01T00:00:00Z
    time_reference_days_since_1950: 25202
    time_reference_julian_day: 2458484.5
    time_reference_seconds_since_1970: 1546300800
    time_coverage_start: 2019-01-01T03:40:34Z
    time_coverage_end: 2019-01-01T04:38:57Z
    time_coverage_duration: PT3503.000S
    time_coverage_resolution: PT1.080S
    orbit: 6310
    references: http://www.tropomi.eu/data-products/nitrogen-dioxide
    processor_version: 1.2.2
    keywords_vocabulary: AGU index terms, http://publications.agu.org/author-resource-center/index-terms/
    keywords: 0345 Pollution, U

In [ ]:
ds.groups

{'METADATA': <class 'netCDF4._netCDF4.Group'>
 group /METADATA:
     dimensions(sizes): 
     variables(dimensions): 
     groups: QA_STATISTICS, ALGORITHM_SETTINGS, GRANULE_DESCRIPTION, ISO_METADATA, EOP_METADATA, ESA_METADATA,
 'PRODUCT': <class 'netCDF4._netCDF4.Group'>
 group /PRODUCT:
     dimensions(sizes): scanline(3245), ground_pixel(450), corner(4), time(1), polynomial_exponents(6), intensity_offset_polynomial_exponents(1), layer(34), vertices(2)
     variables(dimensions): int32 scanline(scanline), int32 ground_pixel(ground_pixel), int32 time(time), int32 corner(corner), int32 polynomial_exponents(polynomial_exponents), int32 intensity_offset_polynomial_exponents(intensity_offset_polynomial_exponents), int32 layer(layer), int32 vertices(vertices), float32 latitude(time, scanline, ground_pixel), float32 longitude(time, scanline, ground_pixel), int32 delta_time(time, scanline), <class 'str'> time_utc(time, scanline), uint8 qa_value(time, scanline, ground_pixel), float32 nitroge

In [ ]:
print(ds.groups['PRODUCT'])

<class 'netCDF4._netCDF4.Group'>
group /PRODUCT:
    dimensions(sizes): scanline(3245), ground_pixel(450), corner(4), time(1), polynomial_exponents(6), intensity_offset_polynomial_exponents(1), layer(34), vertices(2)
    variables(dimensions): int32 scanline(scanline), int32 ground_pixel(ground_pixel), int32 time(time), int32 corner(corner), int32 polynomial_exponents(polynomial_exponents), int32 intensity_offset_polynomial_exponents(intensity_offset_polynomial_exponents), int32 layer(layer), int32 vertices(vertices), float32 latitude(time, scanline, ground_pixel), float32 longitude(time, scanline, ground_pixel), int32 delta_time(time, scanline), <class 'str'> time_utc(time, scanline), uint8 qa_value(time, scanline, ground_pixel), float32 nitrogendioxide_tropospheric_column(time, scanline, ground_pixel), float32 nitrogendioxide_tropospheric_column_precision(time, scanline, ground_pixel), float32 nitrogendioxide_tropospheric_column_precision_kernel(time, scanline, ground_pixel), float32

In [ ]:
print (ds.groups['PRODUCT'].variables.keys())

dict_keys(['scanline', 'ground_pixel', 'time', 'corner', 'polynomial_exponents', 'intensity_offset_polynomial_exponents', 'layer', 'vertices', 'latitude', 'longitude', 'delta_time', 'time_utc', 'qa_value', 'nitrogendioxide_tropospheric_column', 'nitrogendioxide_tropospheric_column_precision', 'nitrogendioxide_tropospheric_column_precision_kernel', 'averaging_kernel', 'air_mass_factor_troposphere', 'air_mass_factor_total', 'tm5_tropopause_layer_index', 'tm5_constant_a', 'tm5_constant_b'])


In [ ]:
ds.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column_precision']

<class 'netCDF4._netCDF4.Variable'>
float32 nitrogendioxide_tropospheric_column_precision(time, scanline, ground_pixel)
    units: mol m-2
    standard_name: troposphere_mole_content_of_nitrogen_dioxide standard_error
    long_name: Precision of the tropospheric vertical column of nitrogen dioxide
    coordinates: longitude latitude
    multiplication_factor_to_convert_to_molecules_percm2: 6.022141e+19
    _FillValue: 9.96921e+36
path = /PRODUCT
unlimited dimensions: 
current shape = (1, 3245, 450)
filling on

In [ ]:
ds.groups['PRODUCT'].variables['longitude'][:][0,:,:][:]

masked_array(
  data=[[-23.17912 , -23.329996, -23.478865, ..., -61.985943, -62.083332,
         -62.18185 ],
        [-23.177786, -23.328894, -23.47799 , ..., -62.035797, -62.13324 ,
         -62.231804],
        [-23.176464, -23.327803, -23.477125, ..., -62.08585 , -62.183342,
         -62.281956],
        ...,
        [ 66.476326,  66.53377 ,  66.59082 , ..., 153.98888 , 154.43105 ,
         154.87555 ],
        [ 66.323326,  66.38029 ,  66.43686 , ..., 154.11232 , 154.55661 ,
         155.00322 ],
        [ 66.16991 ,  66.22638 ,  66.28247 , ..., 154.23701 , 154.68346 ,
         155.13217 ]],
  mask=False,
  fill_value=1e+20,
  dtype=float32)

In [ ]:
no2 = ds.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column_precision'][0,:,:]

In [ ]:
no2

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=9.96921e+36,
  dtype=float32)

In [ ]:
omi_mdf['as_url'].iloc[0]

NameError: ignored

In [ ]:
omi_path = S3Path("s3://drivendata-competition-airathon-public-as/no2/train/omi/2019/", client=s3_cli)
omi_files = list(omi_path.rglob("*.he5"))

In [ ]:
omi_files[0]

In [ ]:
omi_fn = omi_files[0].fspath

In [ ]:
omi_fn

In [ ]:
! pip install rioxarray

In [ ]:
import h5py
import xarray as xr
import rioxarray as rxr

In [ ]:
omi_pre = rxr.open_rasterio(omi_fn, masked=True)

In [ ]:
omi_pre

In [ ]:
omi_pre_qa = omi_pre[0]
omi_pre_qa

In [ ]:
type(omi_pre)